In [4]:
filename = "genes.fasta.txt"
String=""
sequence=""
with open(filename) as diary_file:
  n = 1

  for line in diary_file:
    if line[0]== '>':

      String+= (str(n)+ " " + line)
      n+=1

    else:
      sequence+=line.replace('\n','')
      #print(line.replace('\n',''))
      n += 1  

In [5]:
A=0
C=0
G=0
T=0

for i in sequence:
  if (i=='A' or i=='a'):
    A+=1

  if (i=='C' or i=='c'):
    C+=1
  if (i=='G' or i=='g'):
    G+=1

  if (i=='T' or i=='t'):
    T+=1



In [6]:
mer_counts = {}

for i in range(len(sequence) - 1):
        mer = sequence[i:i+2]
        if mer in mer_counts:
            mer_counts[mer] += 1
        else:
            mer_counts[mer] = 1
    # Return the dictionary with the counts.

In [7]:
print(mer_counts,len(mer_counts))
fin_max = max(mer_counts, key=mer_counts.get)
print(mer_counts[fin_max],fin_max)



{'AC': 39, 'CC': 75, 'CT': 52, 'TT': 26, 'TG': 63, 'GA': 32, 'TA': 12, 'AT': 27, 'TC': 36, 'CA': 53, 'AA': 25, 'AG': 32, 'GG': 40, 'GC': 51, 'GT': 32, 'CG': 21} 16
75 CC


In [8]:
symbols= ['A','C','G','T',fin_max]
frequencies =[A,C,G,T,mer_counts[fin_max]]

In [9]:
import heapq

class Node:
    def __init__(self, symbol, frequency):
        self.symbol = symbol
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency

def huffman_encoding(symbols, frequencies):
    # Create a list of nodes from the symbols and frequencies
    nodes = [Node(s, f) for s, f in zip(symbols, frequencies)]
    
    # Use a min heap to efficiently get the nodes with the lowest frequencies
    heapq.heapify(nodes)
    
    # Create the Huffman tree by continually merging the two nodes with the lowest frequencies
    while len(nodes) > 1:
        left = heapq.heappop(nodes)
        right = heapq.heappop(nodes)
        node = Node(None, left.frequency + right.frequency)
        node.left = left
        node.right = right
        heapq.heappush(nodes, node)
    
    # The final node in the heap is the root of the Huffman tree
    root = nodes[0]
    
    # Use a recursive function to traverse the tree and build the encoding table
    encoding = {}
    def build_encoding(node, prefix=""):
        if node.symbol is not None:
            encoding[node.symbol] = prefix
        else:
            build_encoding(node.left, prefix + "0")
            build_encoding(node.right, prefix + "1")
    build_encoding(root)
    
    return encoding

In [10]:
encoding = huffman_encoding(symbols, frequencies)
print(encoding)

{'T': '00', 'G': '01', 'CC': '100', 'A': '101', 'C': '11'}


In [11]:
# Define the DNA to binary code dictionary
dna_to_binary = encoding

# Define the DNA sequence to be converted
dna_sequence = sequence

# Initialize an empty string to store the binary code
Binary = ''

# Iterate through the DNA sequence
i = 0
while i < len(dna_sequence):
    if i + 1 < len(dna_sequence) and dna_sequence[i:i+2] in dna_to_binary:
        Binary += dna_to_binary[dna_sequence[i:i+2]]
        i += 2
    elif dna_sequence[i] in dna_to_binary:
        Binary += dna_to_binary[dna_sequence[i]]
        i += 1
    else:
        raise ValueError(f"Invalid DNA base or 2-mer: {dna_sequence[i:i+2]}")





In [12]:
R= len(Binary)%8
Extra_Binary= Binary[-R :]
Binary= Binary[0:-R]

In [13]:
def binary_to_ascii(binary_string):
    ascii_string = ""
    for i in range(0, len(binary_string), 8):
        eight_bits = binary_string[i:i+8]
        ascii_string += chr(int(eight_bits, 2))
    return ascii_string



In [14]:
ascii_string = binary_to_ascii(Binary)

In [15]:
print("Compression Ratio is ", len(ascii_string)/len(sequence))

Compression Ratio is  0.2625607779578606


In [16]:
print("****************** writing files **************************")
f = open("Dict.txt", "w")
f.write(str(encoding))
f.close()

****************** writing files **************************


In [17]:
print("****************** writing files **************************")
f = open("Extra_Binary.txt", "w")
f.write(Extra_Binary)
f.close()

****************** writing files **************************


In [18]:
print("****************** writing files **************************")
f = open("compressed.txt",'w')
f.write(ascii_string)
f.close()

****************** writing files **************************


In [19]:
print("****************** writing files **************************")
f = open("meta.txt", "w")
f.write(String)
f.close()

****************** writing files **************************


In [20]:
###### Testing  #################
def ascii_to_binary(ascii_string):
    binary_string = ""
    for char in ascii_string:
        binary_char = bin(ord(char))[2:].zfill(8)
        binary_string += binary_char
    return binary_string

In [21]:
###### Testing  #################
#print((len(ascii_string))
Binary2 = ascii_to_binary(ascii_string)
print(len(Binary2))
print(len(ascii_string))


1296
162


In [22]:
###### Testing  #################

if (Binary == Binary2):
  print("YES")

YES


In [23]:
###### Testing  #################

Binary2+= Extra_Binary

In [24]:
###### Testing  #################

reversedDict = dict()
key_list = list(encoding.keys())
val_list = list(encoding.values())
n = len(key_list)
for i in range(n):
    key = val_list[i]
    val = key_list[i]
    reversedDict[key] = val
print("The reversed dictionary is:")
print(reversedDict)

The reversed dictionary is:
{'00': 'T', '01': 'G', '100': 'CC', '101': 'A', '11': 'C'}


In [25]:
###### Testing  #################

counter = 0
S=""

t=3
A= Binary2

for i in range(0,(len(A)-1)):
  temp = A[counter:counter+t]
  if len(temp) == 1:
    break
  #print(temp)
  if temp in reversedDict: 
    S+=reversedDict[temp]
    #print(temp," ", reversedDict[temp])
    counter+=3
    continue
  temp = A[counter:counter+t-1]
  if temp in reversedDict: 
    S+=reversedDict[temp]
    #print(temp," ", reversedDict[temp])
    #print(temp)
    counter+=2
    continue
  temp = A[counter:counter+t-1-1]
  if temp in reversedDict: 
    S+=reversedDict[temp]
    #print(temp," ", reversedDict[temp])
    #print(temp)
    counter+=1
    continue
    



#print(S)
      



In [26]:
###### Testing  #################

print(len(S))
print(len(sequence))

617
617


In [27]:
###### Testing  #################

if S == sequence :
  print("Yes")

Yes


In [28]:
sequence
print("****************** writing files **************************")
f = open("sequence.txt", "w")
f.write(sequence)
f.close()


****************** writing files **************************


In [29]:
ls

compressed.txt  Extra_Binary.txt  meta.txt      sequence.txt
Dict.txt        genes.fasta.txt   sample_data/


In [30]:
!zip -r /content/lib.zip /content/lib/

	zip warning: name not matched: /content/lib/

zip error: Nothing to do! (try: zip -r /content/lib.zip . -i /content/lib/)
